In [ ]:
from pprint import pprint

import numpy as np
from sklearn.linear_model import LinearRegression

import newron.sklearn

In [ ]:
import newron
from newron.tracking import NewronClient

In [ ]:
remote_server_uri = SERVER_URI # set to your server URI
newron.set_tracking_uri(remote_server_uri)  # or set the MLFLOW_TRACKING_URI in the env
exp_name = "sklearnAutolog"
newron.set_experiment(exp_name)

In [ ]:
def yield_artifacts(run_id, path=None):
    """Yield all artifacts in the specified run"""
    client = NewronClient()
    for item in client.list_artifacts(run_id, path):
        if item.is_dir:
            yield from yield_artifacts(run_id, item.path)
        else:
            yield item.path


def fetch_logged_data(run_id):
    """Fetch params, metrics, tags, and artifacts in the specified run"""
    client = NewronClient()
    data = client.get_run(run_id).data
    # Exclude system tags: https://www.mlflow.org/docs/latest/tracking.html#system-tags
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    artifacts = list(yield_artifacts(run_id))
    return {
        "params": data.params,
        "metrics": data.metrics,
        "tags": tags,
        "artifacts": artifacts,
    }

In [ ]:
newron.sklearn.autolog()

# prepare training data
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([1, 2])) + 3

# train a model
model = LinearRegression()
model.fit(X, y)
run_id = newron.last_active_run().info.run_id
print("Logged data and model in run {}".format(run_id))

# show logged data
for key, data in fetch_logged_data(run_id).items():
    print("\n---------- logged {} ----------".format(key))
    pprint(data)